In [ ]:
import io
import panel as pn
pn.extension()

The ``FileInput`` widget allows uploading one or more file from the frontend and makes the filename, file data and [MIME type](https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/MIME_types) available in Python.

For more information about listening to widget events and laying out widgets refer to the [widgets user guide](../../user_guide/Widgets.ipynb). Alternatively you can learn how to build GUIs by declaring parameters independently of any specific widgets in the [param user guide](../../user_guide/Param.ipynb). To express interactivity entirely using Javascript without the need for a Python server take a look at the [links user guide](../../user_guide/Param.ipynb).

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

##### Core

* **``accept``** (str):  A list of file input filters that restrict what files the user can pick from
* **``filename``** (str/list): The filename(s) of the uploaded file(s)
* **``mime_type``** (str/list): The mime type(s) of the uploaded file(s)
* **``multiple``** (boolean): Whether to allow uploading multiple files
* **``value``** (bytes/list): A bytes object containing the file data or if `multiple` is set a list of bytes objects.

___

In [ ]:
file_input = pn.widgets.FileInput()

file_input

To read out the content of the file you can access the ``value`` parameter, which holds a [bytestring](https://docs.python.org/3/library/stdtypes.html#bytes-objects) containing the file's contents. Additionally information about the file type is made available on the ``filetype`` parameter expressed as a MIME type, e.g. ``image/png`` or ``text/csv``.

In [ ]:
file_input.value

The widget also has a ``save`` method that allows saving the uploaded data to a file or [BytesIO](https://docs.python.org/3/library/io.html#binary-i-o) object.

In [ ]:
# File
if file_input.value is not None:
    file_input.save('test.png')

# BytesIO object
if file_input.value is not None:
    out = io.BytesIO()
    file_input.save(out)

The `accept` parameter restricts what files the user can pick from. This consists of comma-separated list of standard HTML
file input filters. Values can be:

* `<file extension>` - Specific file extension(s) (e.g: .gif, .jpg, .png, .doc) are pickable
* `audio/*` - all sound files are pickable
* `video/*` - all video files are pickable
* `image/*` - all image files are pickable
* `<media type>` - A valid [IANA Media Type](https://www.iana.org/assignments/media-types/media-types.xhtml), with no parameters.

In [ ]:
file_input = pn.widgets.FileInput(accept='.csv,.json')

file_input

To allow uploading multiple files we can also set `multiple=True`:

In [ ]:
file_input = pn.widgets.FileInput(accept='.png', multiple=True)

file_input

When uploading one or more files the `filename`, `mime_type` and `value` parameters will now be lists. 

### Upload size limits

While the `FileInput` widget doesn't set any limit on the size of a file that can be selected by a user, the infrastructure onto which Panel relies (web browsers, Bokeh, Tornado, notebooks, etc.) limits significantly what is actually possible. By default the `FileInput` widget allows to upload data that is in the order of 10 MB. Even if it is possible to increase this limit by setting some parameters (described below), bear in mind that the `FileInput` widget is not meant to upload large files.

#### How it works

Before increasing the file size limit it is worth explaining the process that happens when a file is selected.

The `FileInput` widget is a Bokeh widget whose data is communicated from the front-end to the back-end via a protocal called [Web Sockets](https://en.wikipedia.org/wiki/WebSocket). Bokeh didn't implement Web Sockets itself, instead it took advantage of an existing web framework that provided an implementation: [Tornado](https://www.tornadoweb.org/en/stable/).

In even more concrete terms, here's what happens when a file is selected in a server context (it's the exact same process when multiple files are loaded at once!):

1. The file is loaded into memory by the browser
2. Its content is converted by BokehJS into a [base64 encoded string](https://en.wikipedia.org/wiki/Base64) (which turns a binary file, like a PNG image, into a very long ASCII string)
3. BokehJS puts this long string into a JSON message along with some more information
4. The message is sent through a `Tornado` web socket connection to the back-end
5. The back-end uses it to update the Bokeh/Python model; that's when the properties of the widget in the Bokeh/Python world get updated
6. Panel updates the attributes of the `FileInput` widget instance when the properties of the Python/Bokeh widget are updated. The long string is converted by Panel into a `bytes` object that is available with `.value`.

#### Limits defined

The steps described above almost all have potential or actual limits:

1. Browsers can't upload an unlimited amount of data at once; they're usually limited to a few GB
2. BokehJS uses the [FileReader.readAsDataURL](https://developer.mozilla.org/en-US/docs/Web/API/FileReader/readAsDataURL) method to encode the file as a data URL; browsers can [limit](https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs#common_problems) the length of that URL
3. BokehJS (or the libraries that it uses) may fail at creating the message from a very (very!) large string
4. `Tornado` imposes two limits on the data being transferred: (1) on the maximum size of a buffer and (2) on the maximum size of the websocket message. (1) is 100 MB by default (set by Tornado) and (2) is 20 MB by default (set by Bokeh).

#### Increase the limits

While there's nothing much that can be done about 1., 2. and 3. (except informing your users), the limits defined by Tornado and Bokeh can be overwritten.

##### Server context

In a server context your application must be executed with `python your_app.py` (because `panel serve` doesn't allow to configure all the options provided by Bokeh and Tornado):

```python
# your_app.py
import panel as pn

app = ...

MAX_SIZE_MB = 150

pn.serve(
    app,
    # Increase the maximum websocket message size allowed by Bokeh
    websocket_max_message_size=MAX_SIZE_MB*1024*1014,
    # Increase the maximum buffer size allowed by Tornado
    http_server_kwargs={'max_buffer_size': MAX_SIZE_MB*1024*1014}
)
```

##### Notebook context

In a Jupyter notebook (classic or lab) the limits of Tornado (Tornado's Web Sockets are already used by the Jupyter notebook for communication purposes) can be set in a configuration file. The default maximum buffer size is 512 MB and the default maximum websocked message size is 10 MB.

*Classic Notebook:*

Generate a configuration file with `jupyter notebook --generate-config` and update it with:

```python
c.NotebookApp.tornado_settings = {"websocket_max_message_size": 150 * 1024 * 1024}
c.NotebookApp.max_buffer_size = 150 * 1024 * 1024
```

*Lab:*

Generate a configuration file with `jupyter lab --generate-config` and update it with:

```python
c.ServerApp.tornado_settings = {'websocket_max_message_size': 150 * 1024 * 1024}
c.ServerApp.max_buffer_size = 150 * 1024 * 1024
```

#### Caveats

* The maximum sizes set in either Bokeh or Tornado refer to the maximum size of the **message** that is transferred through the web socket connection, which is going to be larger than the actual size of the uploaded file since the file content is encoded in a `base64` string. So if you set a maximum size of 100 MB for your application, you should indicate your users that the upload limit is a value that is **less** than 100 MB.
* When a file whose size is larger than the limits is selected by a user, their browser/tab may just crash. Alternatively the web socket connection can close (sometimes with an error message printed in the browser console such as `[bokeh] Lost websocket 0 connection, 1009 (message too big)`) which means the application will become unresponsive and needs to be refreshed. 

### Controls

The `FileInput` widget exposes a number of options which can be changed from both Python and Javascript. Try out the effect of these parameters interactively:

In [ ]:
pn.Row(file_input.controls(jslink=True), file_input)